# Batch Training

In [1]:
import yaml
import pandas as pd

# Path to the YAML configuration file
yaml_file_path = 'feature_store/config_v1.yaml'

# Write the configuration data to a YAML file
with open(yaml_file_path, 'r') as file:
    config = yaml.safe_load(file)

In [2]:
# Get Features
from scripts import feature_store
X_train = feature_store.fetch_data_from_store(yaml_file_path = yaml_file_path)
X_train = X_train.head(-7) # save some rows for later
X_train

,lag_1,lag_4,lag_5,lag_6,lag_11,lag_12,lag_13,rolling_mean_7,rolling_std_7
period,,,,,,,,,
2024-01-14,119134.0,129372.0,134119.0,132752.0,131883.0,128866.0,116406.0,127822.29,5524.62
2024-01-15,121604.0,129308.0,129372.0,134119.0,131606.0,131883.0,128866.0,128386.86,6264.72
2024-01-16,136704.0,128467.0,129308.0,129372.0,136960.0,131606.0,131883.0,130031.00,8953.19
2024-01-17,145628.0,119134.0,128467.0,129308.0,130213.0,136960.0,131606.0,133167.71,12008.71
2024-01-18,151329.0,121604.0,119134.0,128467.0,126933.0,130213.0,136960.0,135579.14,12774.85
...,...,...,...,...,...,...,...,...,...
2024-03-27,125493.0,122277.0,130956.0,129138.0,109879.0,119478.0,120293.0,125439.86,4217.72
2024-03-28,125867.0,118230.0,122277.0,130956.0,107680.0,109879.0,119478.0,124931.14,3900.13
2024-03-29,125577.0,126118.0,118230.0,122277.0,118592.0,107680.0,109879.0,123498.29,3072.37


In [3]:
# Get targets
from scripts import feature_processing

csv_file_path = 'data/energy_data_new.csv'
df = pd.read_csv(csv_file_path, parse_dates=['period'])
df.set_index('period', inplace=True)

targets_df = feature_processing.get_targets(df)
targets_df

,target_1d,target_2d,target_3d
period,,,
2024-01-01,128866.0,131883.0,131606.0
2024-01-02,131883.0,131606.0,136960.0
2024-01-03,131606.0,136960.0,130213.0
2024-01-04,136960.0,130213.0,126933.0
2024-01-05,130213.0,126933.0,132752.0
...,...,...,...
2024-03-31,119066.0,124079.0,128889.0
2024-04-01,124079.0,128889.0,127645.0
2024-04-02,128889.0,127645.0,124704.0


In [4]:
feature_store.update_feature_store(targets_df, yaml_file_path, targets = True)

'Targets updated in feature store with last date 2024-04-04 00:00:00'

In [5]:
feature_store.fetch_data_from_store(X_train.index.min(), yaml_file_path, targets = True)


,target_1d,target_2d,target_3d
period,,,
2024-01-14,136704.0,145628.0,151329.0
2024-01-15,145628.0,151329.0,146188.0
2024-01-16,151329.0,146188.0,147359.0
2024-01-17,146188.0,147359.0,144471.0
2024-01-18,147359.0,144471.0,143686.0
...,...,...,...
2024-03-31,119066.0,124079.0,128889.0
2024-04-01,124079.0,128889.0,127645.0
2024-04-02,128889.0,127645.0,124704.0


In [6]:
Y_train = feature_store.fetch_data_from_store(X_train.index.min(), yaml_file_path, targets = True)[:X_train.index.max()]
Y_train

,target_1d,target_2d,target_3d
period,,,
2024-01-14,136704.0,145628.0,151329.0
2024-01-15,145628.0,151329.0,146188.0
2024-01-16,151329.0,146188.0,147359.0
2024-01-17,146188.0,147359.0,144471.0
2024-01-18,147359.0,144471.0,143686.0
...,...,...,...
2024-03-27,125577.0,120926.0,113774.0
2024-03-28,120926.0,113774.0,107790.0
2024-03-29,113774.0,107790.0,119066.0


In [7]:
# turn into a list
y_train_ls= []
for target in Y_train.columns:
  y_train_ls.append(Y_train[target])

y_train_ls

[period
 2024-01-14    136704.0
 2024-01-15    145628.0
 2024-01-16    151329.0
 2024-01-17    146188.0
 2024-01-18    147359.0
                 ...   
 2024-03-27    125577.0
 2024-03-28    120926.0
 2024-03-29    113774.0
 2024-03-30    107790.0
 2024-03-31    119066.0
 Name: target_1d, Length: 78, dtype: float64,
 period
 2024-01-14    145628.0
 2024-01-15    151329.0
 2024-01-16    146188.0
 2024-01-17    147359.0
 2024-01-18    144471.0
                 ...   
 2024-03-27    120926.0
 2024-03-28    113774.0
 2024-03-29    107790.0
 2024-03-30    119066.0
 2024-03-31    124079.0
 Name: target_2d, Length: 78, dtype: float64,
 period
 2024-01-14    151329.0
 2024-01-15    146188.0
 2024-01-16    147359.0
 2024-01-17    144471.0
 2024-01-18    143686.0
                 ...   
 2024-03-27    113774.0
 2024-03-28    107790.0
 2024-03-29    119066.0
 2024-03-30    124079.0
 2024-03-31    128889.0
 Name: target_3d, Length: 78, dtype: float64]